# Cytoscape Reactome FI Example

Overview
--------
This example demonstrates a Cytoscape Reactome FI workflow
for the following cohorts:

* BRCA (Breast invasive carcinoma)

* OV (Ovarian serous cystadenocarcinoma)

The steps executed are as follows:

1. Download each cohort mutation annotation file (MAF) from the
   Broad Firehose WS server.

2. Build the two networks using the CyREST API `ReactomeFIViz`
   `buildFISubNetwork` command.

3. Cluster the network into gene modules using the `ReactomeFIViz`
   `cluster` command. 

4. Perform pair-wise module overlapping analysis.

5. Load the Reactome pathways.

6. Partition a subset of the gene modules into the following sets:
   
   * Modules with a significant proportion of genes in common
   
   * Unshared modules for each of the two cohorts

7. Perform pathway enrichment analysis for each unshared gene module.

8. Export unshared gene pathway diagrams.

Prerequisites
------------
* Python version 2.6 or higher

* [fbget](https://confluence.broadinstitute.org/display/GDAC/fbget)

* Cytoscape with the latest version of ReactomeFIViz

Install firebrowse per the [fbget Download](https://confluence.broadinstitute.org/display/GDAC/fbget#fbget-configuration) instructions. Include both the `firebrowse` package and the `fbget.py` file in your Python path. For example, in the [Anaconda](https://docs.anaconda.com/anaconda/) virtual environment:

    mkdir -p /usr/local/src/cytoscape
    (cd /usr/local/src/cytoscape;
     curl -o fbget-0.1.11.zip http://gdac.broadinstitute.org/runs/code/fbget-0.1.11.zip;
     unzip fbget-0.1.11.zip;
     mv fbget-0.1.11/fbget.py .;
     rm -r fbget-0.1.11*
    )
    conda create -n cytoscape pip jupyter pandas scipy statsmodels
    source activate cytoscape
    pip install firebrowse
    conda develop /usr/local/src/cytoscape

Open Cytoscape before building the networks below. Select `Help > Check for Updates` to ensure that ReactomeFIViz is current.

*Note*: Building the networks clears an existing Cytoscape session, if it exists.

In [1]:
# The Cytoscape download module.
import fbget
# Python utilities.
import os
import requests
import tempfile
import shutil
import itertools
from io import StringIO
import pandas as pd
from scipy.stats import hypergeom
import numpy as np
from statsmodels.sandbox.stats.multicomp import multipletests

In [2]:
sandbox = tempfile.mkdtemp()
print "The downloaded files will be saved to %s." % sandbox

The downloaded files will be saved to /var/folders/20/lm3r8z697lz5wskmqzfkkjv482fhg1/T/tmpdTt9F9.


In [3]:
# The REST service url.
service_url = 'http://localhost:1234'
# Returns the REST app url.
def get_url(*params, **kwargs):
    path = [service_url]
    app = kwargs.get('app')
    if app is not None:
        path.append(app)
    path.append('v1')
    path.extend(params)
    return '/'.join(path)

# Returns the CyREST Reactome FI url.
get_fi_url = lambda *params: get_url(*params, app='reactomefiviz')

In [495]:
# Returns the data frame for the given CyREST Reactome
# FI response.  The response must contain a 'data'
# property object with content
# {tableHeaders, tableHeaders, tableContent}.
# The required *parsers* dictionary argument associates
# a parser with each column.
# The optional index parameter is the index column name.
def parse_fi_table_response(resp, parsers, index=None):
    # The cluster response data.
    data = resp.json()['data']
    # The cluster data columns.
    columns = data['tableHeaders']
    parsers_list = [parsers[col] for col in columns]
    # Parses a cluster content row.
    parse_row = lambda row: tuple(parsers_list[i](value) for i, value in enumerate(row))
    # The parsed cluster content list.
    content = map(parse_row, data['tableContent'])
    # Return the cluster data frames.
    return pd.DataFrame.from_records(content, index=index, columns=columns)

In [217]:
# Downloads the MAF file for the given cohort.
def download_maf(cohort, out=None):
    # Download the MAF.
    maf_file = out if out else os.path.join(sandbox, "%s.maf" % cohort) 
    print "Downloading the %s MAF file to %s..." % (cohort, maf_file)
    maf_csv = fbget.maf(cohort=cohort)
    # Parse the MAF as a data frame.
    maf_df = pd.read_csv(StringIO(maf_csv), sep='\t', index_col='Module')
    # Write the MAF file.
    maf_df.to_csv(maf_file, index=False, sep='\t')
    return maf_file

In [235]:
# Builds the network.
def build_network(maf_file):
    # Clear the current Cytoscape session, if any.
    requests.delete(get_url('session')).ok
    # Read the MAF into a data frame.
    maf_df = pd.read_csv(maf_file, sep='\t')
    # The number of samples.
    sample_cnt = len({tsb for tsb in maf_df.Tumor_Sample_Barcode})
    print "Sample Count: %d" % sample_cnt
    sample_cutoff = max(2, int(0.01*sample_cnt))
    print "Building the FI network with sample cut-off %d..." % sample_cutoff
    # Build the FI network with 1% sample cut-off.
    body = dict(fiVersion='2016', format='MAF', file=maf_file,
                enteredGenes='', chooseHomoGenes=False,
                userLinkers=False, showUnLinked=False,
                fetchFIAnnotations=True,
                sampleCutoffValue=sample_cutoff)
    requests.post(get_fi_url('buildFISubNetwork'), json=body)
    print "The FI network is loaded to Cytoscape."
    return maf_df

In [28]:
# The cluster content value parsers.
cluster_parsers = {
    'Module': int, 'Nodes in Module': int, 'Node Percentage': float,
    'Samples in Module': int, 'Sample Percentage': float,
    'Node List': lambda s: s.split(',')
}

# Parses the clusters into data frames.
def cluster():
    # Cluster the currently displayed network.
    print "Clustering the FI network..."
    resp = requests.get(get_fi_url('cluster'))
    # Parse the response JSON into a data frame.
    df = parse_fi_table_response(resp, parsers=cluster_parsers, index='Module')
    print "The cluster table is available in Cytoscape."
    return df

In [32]:
# Prepares the given cohort for analysis.
# Returns the clustered gene modules.
def prepare(cohort):
    # Clear the current Cytoscape session, if any.
    requests.delete(get_url('session')).ok
    maf_file = os.path.join(sandbox, "%s.maf" % cohort)
    # Download the MAF, if necessary.
    if (not os.path.exists(maf_file)):
        download_maf(cohort, out=maf_file)
    # Build the network.
    build_network(maf_file)
    # Cluster into gene modules.
    return cluster()

In [353]:
# Returns the probability of gene set overlap from
# a background population of N genes.
def overlap_pvalue(N, gs1, gs2):
    # The shared genes.
    overlap = set(gs1).intersection(set(gs2))
    M = len(overlap)
    n1 = len(gs1)
    n2 = len(gs2)
    # The CDF of complete overlap.
    kmax = min(n1, n2)
    cdf_max = hypergeom.cdf(kmax, N, n1, n2)
    # The CDF of the observed overlap.
    kmin = M - 1
    cdf_obs = hypergeom.cdf(kmin, N, n1, n2)
    #print ("%d %d %d %d %d %0.5f %0.5f" % (M, n1, n2, kmax, kmin, cdf_max, cdf_obs))
    # Return the overlap probability.
    return cdf_max - cdf_obs

In [264]:
# Returns module sets with module count no greater than a
# maximum size given by the *max_size* option with default
# the smallest module gene count.
def filter_on_size(*modules, **kwargs):
    n = kwargs.get('max_size', min(len(df['Nodes in Module']) for df in modules))
    largest = [df.nlargest(n, 'Nodes in Module') for df in modules]
    cutoff = max(df.iloc[n - 1]['Nodes in Module'] for df in largest)
    criterion = lambda df: df['Nodes in Module'] >= cutoff
    return [df.loc[criterion, :] for df in largest]

# Slices the given data frame vertically on the 'Node List' column.
# Renames the 'Module' index to the given name and the 'Node List'
# column to <name> Genes, e.g. 'BRCA Genes' for *name* 'BRCA'.
def slice_on_node_list(df, name):
    rename_dict = {'Node List': "%s Genes" % name}
    sliced = df.loc[:, ['Node List']].rename(columns=rename_dict)
    sliced.index.names = [name]
    return sliced

# Returns the cartesian product of two data frames.
def cartesian(df1, df2):
    rows = itertools.product(df1.iterrows(), df2.iterrows())
    values = (left.append(right) for (_, left), (_, right) in rows)
    indexes = [df1.index, df2.index]
    index_names = [index.name for index in indexes]
    index_values = [index.values for index in indexes]
    multi_index = pd.MultiIndex.from_product(index_values, names=index_names)
    return pd.DataFrame(values, index=multi_index)

In [374]:
# The number of background genes N is the Reactome FI
# network gene size.
N = 12283

# Calculates the pair-wise overlap p-value for the given
# gene module cross-product.
# The return value is the cross-product augmented with
# two columns:
# * _uncorrected_pvalue_ - the hypergeometric test p value
# * _corrected_pvalue_ - the p-values corrected for multiple tests
def append_overlap(cross):
    # Calculates the overlap p-value.
    pvals = cross.agg(lambda row: overlap_pvalue(N, *row.values), axis=1)
    # Augment the cross-product with the uncorrrected p-values.
    overlap = cross.assign(uncorrected_pvalue=pvals)
    # Correct the given p-values for multiple comparison hypothesis
    # testing by applying the Benjamini–Hochberg FDR procedure.
    _, corrected, _, _ = multipletests(pvals.values, method='fdr_bh')
    overlap['corrected_pvalue'] = corrected
    return overlap

# Partitions the genes into three sets:
# * Shared gene sets for modules with significant overlap
# * Unshared BRCA modules
# * Unshared OV modules
# The input data frame is the BRCA x OV module cartesian
# cross-product augmented with a pair-wise corrected
# overlap p-value.
# The corrected p-value *cutoff* determines a shared module.
def partition_shared(cross, cutoff=0.01):
    # The shared gene modules have corrected p-value < cutoff
    criterion = df['corrected_pvalue'].lt(.01)
    shared = overlap.loc[lambda df: df['corrected_pvalue'].lt(.01)]    

In [406]:
# Performs pair-wise overlap analysis on the given clusters.
# Returns the BRCA x OV module cross-product data frame
# augmented with *uncorrected_pvalue* and *corrected_pvalue*
# columns.
def analyse_overlap(brca, ov):
    # Select the 20 or so largest modules.
    brca_filtered, ov_filtered = filter_on_size(brca, ov, max_size=20)
    # Prep pair-wise analysis by taking the cartesian product
    # of the two filtered data frames renamed with unique columns.
    brca_slice = slice_on_node_list(brca_filtered, 'BRCA')
    ov_slice = slice_on_node_list(ov_filtered, 'OV')
    cross = cartesian(brca_slice, ov_slice)
    overlap = append_overlap(cross)
    return overlap

In [496]:
enrichment_parsers = {
    'ReactomePathway': lambda p: p,
    'RatioOfProteinInPathway': float,
    'NumberOfProteinInPathway': int,
    'ProteinFromGeneSet': int,
    'P-value': float,
    'FDR': float,
    'HitGenes': lambda s: s.split(',')
}

# Perform pathway enrichment analysis on the given gene list or set.
# Returns the result data frame.
def enrich(genes):
    data = ','.join(genes)
    # Perform the Reactome enrichment analysis.
    resp = requests.post(get_fi_url('ReactomePathwayEnrichment'), json=data)
    # Return the data frames.
    return parse_fi_table_response(resp, enrichment_parsers, index='ReactomePathway')

In [236]:
# Prepare the BRCA cohort for analysis.
brca_df = prepare('BRCA')

Sample Count: 978
Building the FI network with sample cut-off 9...
The FI network is loaded to Cytoscape.
Clustering the FI network...
The cluster table is available in Cytoscape.


*Expected Result*: Cytoscape displays the BRCA FI network
and clustered gene modules table.

In [237]:
# Prepare the OV cohort for analysis.
ov_df = prepare('OV')

Sample Count: 466
Building the FI network with sample cut-off 4...
The FI network is loaded to Cytoscape.
Clustering the FI network...
The cluster table is available in Cytoscape.


*Expected Result*: Cytoscape displays the OV FI network
and clustered gene modules table.

In [407]:
# Discover modules with significant overlap.
overlap = analyse_overlap(brca_df, ov_df)
# Filter by 1% FPR.
criterion = overlap['corrected_pvalue'].lt(.01)
shared = overlap.loc[criterion]
# Print summary counts.
print "Cluster BRCA module count: %d" % len(brca_df)
print "Cluster OV module count: %d" % len(ov_df)
print "Filtered BRCA module count: %d" % len(overlap.groupby('BRCA'))
print "Filtered OV module count: %d" % len(overlap.groupby('OV'))
print "Significant overlap pairs count: %d" % len(shared)
print "Shared BRCA module count: %d" % len(shared.groupby('BRCA'))
print "Shared OV module count: %d" % len(shared.groupby('OV'))

Cluster BRCA module count: 23
Cluster OV module count: 38
Filtered BRCA module count: 18
Filtered OV module count: 20
Significant overlap pairs count: 24
Shared BRCA module count: 16
Shared OV module count: 14


In [408]:
# Load Reactome pathways.
resp = requests.get(get_fi_url('pathwayTree'))
reactome_tree = resp.json()['data']

*Expected Result*: Reactome heirarchy displayed in the Cytoscape Control Panel.

In [525]:
# Perform pathway enrichment on the shared gene modules.
genelists = shared.loc[:, 'BRCA Genes':'OV Genes'].values
genesets = [set(brca).union(set(ov)) for brca, ov in genelists]
enrichments = [enrich(gs) for gs in genesets]
filtered = [e.FDR.loc[e.FDR.lt(.001)] for e in enrichments]
print "Example significant shared pathways:"
filtered[0]

Example significant shared pathways:


ReactomePathway
Generic Transcription Pathway                                                                     2.229000e-11
Transcriptional regulation of white adipocyte differentiation                                     1.595200e-10
Chromatin modifying enzymes                                                                       1.917500e-10
Regulation of lipid metabolism by Peroxisome proliferator-activated receptor alpha (PPARalpha)    5.778800e-10
PPARA activates gene expression                                                                   3.972800e-09
Fatty acid, triacylglycerol, and ketone body metabolism                                           3.972800e-09
RORA activates gene expression                                                                    5.772900e-08
YAP1- and WWTR1 (TAZ)-stimulated gene expression                                                  8.725300e-08
Transcriptional activation of mitochondrial biogenesis                                          

*Expected Result*: Enrichment results are successively displayed in the Cytoscape Table Panel.

In [ ]:
# TODO - Export the diagram for one pathway.

In [ ]:
# TODO - Move doc cells below to appropriate steps above.

(cf. User Guide
[Explore Pathways](http://wiki.reactome.org/index.php?title=ReactomeFIViz&redirect=no#Explore_Reactome_Pathways)
step)

(cf. User Guide
[Enrichment Analysis](http://wiki.reactome.org/index.php?title=ReactomeFIViz&redirect=no#Pathway_Enrichment_Analysis)
step)

(cf. User Guide
[Mutation Analysis](http://wiki.reactome.org/index.php?title=ReactomeFIViz&redirect=no#Gene_Set.2FMutation_Analysis)
step)